# Project 2 - Source coding, data compression and channel coding

The goal of this second project is to apply some of the principles seen in the lectures about source coding, data compression and channel coding. We ask you to write a brief report (pdf format) collecting your answers to the different questions. All codes must be written in Python inside this Jupyter. Note that you can not change the content of locked cells or import any extra Python library than the ones already imported (numpy).

## Implementation

In this project, you will need to use implement source coding algorithms to answer several questions. Therefore, in this first part, you are asked to write several functions that implement two of the  algorithms seen in the theoretical lectures and one new algorithm described in the project statement. Remember that you need to fill in this Jupyter Notebook to answer these questions. Pay particular attention to the required input and output format of each function.

In [3]:
# [Locked Cell] You can not import any extra Python library in this Notebook.
import numpy as np

### Question 1
Implement a function that returns a binary Huffman code for a given probability distribution. Give the main steps of your implementation. Verify your code on Exercise 7 of the second exercise session (TP2), and report the output of your code for this example. Explain how to extend your function to generate a Huffman code of any (output) alphabet size. 


In [25]:
class Node:
    """
    Node class for Huffman tree
    """
    def __init__(self, probability, symbol, leftChild=None, rightChild=None):
        self.probability = probability
        self.leftChild = leftChild
        self.rightChild = rightChild
        self.symbol = symbol

def get_codeword(node, codeword):
    """
    Recursively get the codeword for each node in the Huffman tree
    
    Arguments:
    ----------
    node: root node of the Huffman tree
    codeword: codeword already built (accumulator)
    
    Return:
    -------
    - codewords: dictionary with the name and the corresponding codeword 
      - keys: symbol as character or string
      - values: associated codeword as a character or a string    
      Example: {"A": "10", "B":"0","C":"111","D":"110"}
    
    """
    if node.leftChild is None and node.rightChild is None:
        return {node.symbol: codeword}
    else:
        codeword_left = codeword + "0"
        codeword_right = codeword + "1"
        codewords = get_codeword(node.leftChild, codeword_left)
        codewords.update(get_codeword(node.rightChild, codeword_right))
        return codewords

def Huffman_code(probability_dict):
    """
    Create the Huffman code for given probabilities  
    
    Arguments:
    ----------
    probability_dict:
      - keys: symbol as character or string
      - values: probability of the symbol as a float or double
      Example: {"A": 0.25, "B":0.5,"C":0.125,"D":0.125}
    
    Return:
    -------
    - codewords: dictionary with the name and the corresponding codeword 
      - keys: symbol as character or string
      - values: associated codeword as a character or a string    
      Example: {"A": "10", "B":"0","C":"111","D":"110"}
    
    """ 
    # Create the nodes of the tree from the probabilities
    nodes = []
    for symbol, probability in probability_dict.items():
        nodes.append(Node(probability, symbol))

    # Sort the list of Node objects by probability
    nodes.sort(key=lambda x: x.probability)

    # Create the Huffman tree
    while len(nodes) > 1:
        left = nodes.pop(0)
        right = nodes.pop(0)
        parent = Node(left.probability + right.probability, None, left, right)
        nodes.append(parent)
        nodes.sort(key=lambda x: x.probability)

    # Get the codewords on the root of the tree
    codewords = get_codeword(nodes[0], "")
    return codewords

# Verification on the exerice 7 of the second exercise session (TP2)
print("Code word of the alphabet of the exercice 7 of the second exercise session (TP2): {}".format(Huffman_code({"A": 0.05, "B":0.1,"C":0.15,"D":0.15, "E":0.2, "F":0.35})))

Code word of the alphabet of the exercice 7 of the second exercise session (TP2): {'A': '000', 'B': '001', 'E': '01', 'C': '100', 'D': '101', 'F': '11'}


### Question 2

Given a sequence of symbols, implement a function that returns a dictionary and the encoded sequence using the on-line Lempel-Ziv algorithm (see State of the art in data compression, slide 50/53). Reproduce and report the example given in the course. 

In [51]:
def get_least_significant_bit(number, bitNumber):
    """
    Get the least significant bit of a number and convert it to a binary string with bitNumber digits
    Example: get_least_significant_bit(1011, 2) -> "01"
    """
    bit = number[-1]
    return "0" * (bitNumber - 1) + bit

def get_first_bits(number):
    """
    Get the all except the last bit of a binary number represented as a string
    Example: get_first_bits('101') -> '10'
    """
    return number[0:len(number)-1]

def get_binary_representation(number, bitNumber):
    """
    Get the binary representation of a number
    Example: get_binary_representation(5, 4) -> '0101'
    """
    return bin(number)[2:].zfill(bitNumber)

def LZ_online(sequence):
    """
    The on-line Lempel-Ziv algorithm given a sequence of symbols  
    Arguments:
    ----------
    - sequence : Sequence of symbols in the string format
    
    Return:
    -------
    - dictionary : the computed dictionnary in the form:
        - keys: symbol as character or string
        - values: associated codeword as a tuple composed of the entry index (integer) and a binarized address with one appended symbol (character or string)
        Example: {'': (0, ''), '0': (1, '0'), '1': (2, '01'), '00': (3, '010'), '10': (4, '100')}
    - encoded_sequence : the encoded sequence in the string format
    """
    # Initialization of the dictionary
    dictionnary = {}
    dictionnary[''] = (0, '')

    prefix = ''
    position = 1

    encoded_sequence = ''
    # Build the dictionary and encode the sequence
    for symbol in sequence:
        if prefix + symbol not in dictionnary:
            bitNumber = int(np.ceil(np.log2(len(dictionnary))) + 1)
            if len(prefix + symbol) == 1:
                dictionnary[prefix + symbol] = (position, get_least_significant_bit(prefix + symbol, bitNumber))
            else: 
                dictionnary[prefix + symbol] = (position, get_binary_representation(dictionnary[get_first_bits(prefix + symbol)][0], bitNumber - 1) + get_least_significant_bit(prefix + symbol, 1))
            encoded_sequence += dictionnary[prefix + symbol][1]
            prefix = ''
            position += 1
        else:
            prefix += symbol

    return dictionnary, encoded_sequence

print("Encoding of example given in the course State of the art in data compression, slide 50/53: {}".format(LZ_online("1011010100010")))

: ({'': (0, ''), '1': (1, '1'), '0': (2, '00'), '11': (3, '011'), '01': (4, '101'), '010': (5, '1000'), '00': (6, '0100'), '10': (7, '0010')}, '100011101100001000010')


### Question 4

Implement a function that returns the encoded sequence using the LZ77 algorithm as described by the algorithm below given an input string and a sliding window size l. Reproduce the example given in Figure 2 with window_size=7.

In [ ]:
def LZ77(sequence, window_size=7):
    """
    The Lempel-Ziv 77 algorithm given a sequence of symbols and the sliding window size
    
    Arguments:
    ----------
    - sequence : Sequence of symbols in the string format
    - window_size : sliding window size as an integer
    
    Return:
    -------
    - encoded_sequence : the encoded sequence in the string format
    """

In [ ]:
# [Locked Cell] Evaluation of your functions by the examiner. 
# You don't have access to the evaluation, this will be done by the examiner.
# Therefore, this cell will return nothing for the students.
import os
if os.path.isfile("private_evaluation.py"):
    from private_evaluation import unit_tests
    unit_tests(Huffman_code, LZ_online, LZ77)

## Source coding and reversible (lossless) data compression


In [ ]:
# Write here your codes for questions 5 to 15 (you may delete this comment)

## Channel coding

In [ ]:
# Write here your codes for questions 16 to 21 (you may delete this comment)
# From here, you may import either opencv (cv2) or the Python Imaging Library (PIL), but no other extra libraries.